In [ ]:
!pip install pydub
!apt-get install ffmpeg -y
!pip install soundfile librosa numpy noisereduce
!pip install speechbrain torchaudio
!pip install -U openai-whisper
!pip install torch torchvision torchaudio

In [ ]:
from pydub import AudioSegment
from pathlib import Path

# Input and output file paths
input_file = "/content/audio(1).mp3"
output_file = "/content/audio.wav"

# Automatically detect input format and convert to WAV
input_path = Path(input_file)
audio = AudioSegment.from_file(input_file, format=input_path.suffix[1:].lower())
audio.export(output_file, format="wav")

print(f"Conversion completed. WAV file saved to: {output_file}")

In [ ]:
import os
import warnings
import numpy as np
import librosa
import soundfile as sf
import scipy.signal
from typing import Tuple

warnings.filterwarnings("ignore")


class AudioPreprocessor:
    def __init__(self, target_sr: int = 16000):
        """Initialize the preprocessor with a target sample rate."""
        self.target_sr = target_sr

    def load_audio(self, path: str) -> Tuple[np.ndarray, int]:
        """Load audio from a file."""
        try:
            audio, sr = librosa.load(path, sr=None)
        except Exception:
            audio, sr = sf.read(path)
            if len(audio.shape) > 1:
                audio = np.mean(audio, axis=1)
        return audio, sr

    def resample(self, audio: np.ndarray, sr: int) -> np.ndarray:
        """Resample audio to the target sample rate."""
        if sr != self.target_sr:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=self.target_sr)
        return audio

    def normalize(self, audio: np.ndarray, method: str = 'peak') -> np.ndarray:
        """Normalize audio signal."""
        if method == 'peak':
            peak = np.max(np.abs(audio))
            if peak > 0:
                audio = 0.95 * audio / peak
        elif method == 'rms':
            rms = np.sqrt(np.mean(audio**2))
            if rms > 0:
                audio = audio * (0.1 / rms)
        elif method == 'lufs':
            loudness = np.mean(np.abs(audio))
            if loudness > 0:
                audio = audio * (0.1 / loudness)
        return np.clip(audio, -1.0, 1.0)

    def highpass_filter(self, audio: np.ndarray, cutoff: float = 80.0) -> np.ndarray:
        """Apply high-pass filter to remove low-frequency noise."""
        nyquist = self.target_sr / 2
        b, a = scipy.signal.butter(4, cutoff / nyquist, btype='high')
        return scipy.signal.filtfilt(b, a, audio)

    def spectral_subtraction(self, audio: np.ndarray) -> np.ndarray:
        """Reduce noise via spectral subtraction."""
        stft = librosa.stft(audio, n_fft=2048, hop_length=512)
        mag, phase = np.abs(stft), np.angle(stft)
        noise_est = np.mean(mag[:, :min(10, mag.shape[1] // 4)], axis=1, keepdims=True)
        cleaned_mag = np.maximum(mag - 2.0 * noise_est, 0.1 * mag)
        enhanced = librosa.istft(cleaned_mag * np.exp(1j * phase), hop_length=512)
        return enhanced

    def remove_silence(self, audio: np.ndarray) -> np.ndarray:
        """Trim leading and trailing silence from the audio."""
        audio_trimmed, _ = librosa.effects.trim(audio, top_db=30)
        return audio_trimmed

    def preprocess_audio(
        self,
        audio: np.ndarray,
        sr: int,
        normalize_method: str = 'peak',
        denoise_method: str = 'spectral_subtraction',
        apply_highpass: bool = True,
        remove_silence_flag: bool = False
    ) -> np.ndarray:
        """Apply full preprocessing pipeline to audio."""
        audio = self.resample(audio, sr)
        if remove_silence_flag:
            audio = self.remove_silence(audio)
        if apply_highpass:
            audio = self.highpass_filter(audio)
        if denoise_method == 'spectral_subtraction':
            audio = self.spectral_subtraction(audio)
        audio = self.normalize(audio, method=normalize_method)
        return audio

    def process_file(
        self,
        input_path: str,
        output_path: str,
        normalize_method: str = 'peak',
        denoise_method: str = 'spectral_subtraction'
    ):
        """Process a single audio file from input to output."""
        audio, sr = self.load_audio(input_path)
        processed_audio = self.preprocess_audio(
            audio,
            sr,
            normalize_method=normalize_method,
            denoise_method=denoise_method,
            apply_highpass=True,
            remove_silence_flag=False
        )
        sf.write(output_path, processed_audio, self.target_sr)
        print(f"✅ Processed: {input_path} → {output_path}")


# === Example Usage ===
if __name__ == "__main__":
    # Define input and output file paths
    input_file = "/content/audio.wav"
    output_file = "/content/preprocess.wav"

    # Create preprocessor and run
    preprocessor = AudioPreprocessor(target_sr=16000)
    preprocessor.process_file(
        input_path=input_file,
        output_path=output_file,
        normalize_method='rms',              # Options: 'peak', 'rms', 'lufs'
        denoise_method='spectral_subtraction'  # Options: 'spectral_subtraction', 'wiener', None
    )


In [ ]:
import os
import torch
import torchaudio
import numpy as np
import soundfile as sf
from speechbrain.pretrained import VAD

class SpeechActivityDetector:
    def __init__(self, model_dir="speechbrain/vad-crdnn-libriparty", device=None):
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = device
        self.vad = VAD.from_hparams(source=model_dir, run_opts={"device": device})
        self.sample_rate = 16000

    def load_audio(self, path):
        waveform, sr = torchaudio.load(path)
        if sr != self.sample_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.sample_rate)
        return waveform[0]

    def get_speech_segments(self, signal, threshold=0.5, min_duration=0.25):
        signal = signal.to(self.device)
        with torch.no_grad():
            probs = self.vad.forward(signal.unsqueeze(0))  # Shape: [1, T]

        probs = probs.squeeze().cpu().numpy()
        frame_duration = 0.01  # 10 ms per frame

        segments = []
        start = None

        for i, p in enumerate(probs):
            t = i * frame_duration
            if p > threshold:
                if start is None:
                    start = t
            else:
                if start is not None:
                    end = t
                    if end - start >= min_duration:
                        segments.append((start, end))
                    start = None

        if start is not None:
            end = len(probs) * frame_duration
            if end - start >= min_duration:
                segments.append((start, end))

        return segments

    def save_segments(self, signal, segments, output_dir, basename="segment"):
        os.makedirs(output_dir, exist_ok=True)
        for i, (start, end) in enumerate(segments):
            start_sample = int(start * self.sample_rate)
            end_sample = int(end * self.sample_rate)
            segment = signal[start_sample:end_sample]
            filename = f"{basename}_{i:03d}_{start:.2f}_{end:.2f}.wav"
            sf.write(os.path.join(output_dir, filename), segment.cpu().numpy(), self.sample_rate)

    def detect_and_save(self, audio_path, output_dir):
        print(f"Processing {audio_path}")
        signal = self.load_audio(audio_path)
        segments = self.get_speech_segments(signal)
        self.save_segments(signal, segments, output_dir, basename=os.path.splitext(os.path.basename(audio_path))[0])
        print(f"Saved {len(segments)} segments to {output_dir}")
        return segments


# Example usage
if __name__ == "__main__":
    audio_file = "/content/preprocess.wav"
    output_dir = "/content/vad_segments"

    sad = SpeechActivityDetector()
    segments = sad.detect_and_save(audio_file, output_dir)

    for i, (start, end) in enumerate(segments):
        print(f"Segment {i}: {start:.2f}s - {end:.2f}s")

In [ ]:
import os
import librosa
import soundfile as sf
import math

def segment_audio_librosa_with_overlap(
    input_folder, output_folder,
    window_duration_sec=2.0, overlap_sec=0.5, sample_rate=16000
):
    os.makedirs(output_folder, exist_ok=True)
    window_samples = int(window_duration_sec * sample_rate)
    hop_samples = int((window_duration_sec - overlap_sec) * sample_rate)

    global_chunk_id = 1

    for filename in sorted(os.listdir(input_folder)):
        if not filename.lower().endswith(".wav"):
            continue

        parts = filename.split('_')
        try:
            segment_start_time = float(parts[2])
            segment_end_time = float(parts[3].replace('.wav', ''))
        except (IndexError, ValueError):
            print(f"Warning: Could not parse timestamp from {filename}. Using 0.0 as start time.")
            segment_start_time = 0.0

        print(f"Processing {filename}, start time: {segment_start_time}, end time: {segment_end_time}")

        filepath = os.path.join(input_folder, filename)
        y, sr = librosa.load(filepath, sr=sample_rate)
        total_samples = len(y)

        # Calculate number of windows with overlap
        if total_samples <= window_samples:
            num_windows = 1
        else:
            num_windows = 1 + math.ceil((total_samples - window_samples) / hop_samples)

        for i in range(num_windows):
            start = i * hop_samples
            end = start + window_samples
            if end > total_samples:
                end = total_samples
                start = max(0, end - window_samples)  # Shift start if last window is shorter

            chunk = y[start:end]

            chunk_start_time = segment_start_time + (start / sample_rate)
            chunk_end_time = segment_start_time + (end / sample_rate)

            chunk_filename = f"segment{global_chunk_id}_{chunk_start_time:.2f}_{chunk_end_time:.2f}.wav"
            chunk_path = os.path.join(output_folder, chunk_filename)
            sf.write(chunk_path, chunk, sample_rate)

            global_chunk_id += 1

        print(f"Segmented {filename} into {num_windows} overlapping chunks.")

# Example usage
input_folder = "/content/vad_segments"
output_folder = "/content/segments_overlap"
segment_audio_librosa_with_overlap(input_folder, output_folder, window_duration_sec=2.0, overlap_sec=0.5)

In [ ]:
import os
import torch
import torchaudio
import numpy as np
from speechbrain.pretrained import EncoderClassifier
import pickle
import json
from pathlib import Path
from tqdm import tqdm
import argparse

class SpeakerEmbeddingExtractor:
    def __init__(self, model_name="speechbrain/spkrec-ecapa-voxceleb"):
        """Initialize ECAPA-TDNN speaker embedding extractor"""
        print("Loading ECAPA-TDNN model...")
        self.classifier = EncoderClassifier.from_hparams(
            source=model_name,
            savedir="pretrained_models/spkrec-ecapa-voxceleb"
        )
        print("Model loaded successfully!")

    def extract_embedding(self, audio_path):
        """Extract speaker embedding from an audio file"""
        try:
            signal, fs = torchaudio.load(audio_path)

            if signal.shape[1] < 16000:  # less than 1 sec
                print(f"[SKIP] Too short: {audio_path}")
                return None

            if fs != 16000:
                resampler = torchaudio.transforms.Resample(fs, 16000)
                signal = resampler(signal)

            if signal.shape[0] > 1:
                signal = torch.mean(signal, dim=0, keepdim=True)

            with torch.no_grad():
                embedding = self.classifier.encode_batch(signal)
                embedding = embedding.squeeze().cpu().numpy()

            if embedding.size == 0 or np.any(np.isnan(embedding)):
                print(f"[SKIP] Invalid embedding: {audio_path}")
                return None

            return embedding

        except Exception as e:
            print(f"[ERROR] {audio_path}: {str(e)}")
            return None

    def process_folder(self, input_folder, output_folder, audio_extensions=None):
        """Process all audio files in a folder and save embeddings"""
        if audio_extensions is None:
            audio_extensions = ['.wav', '.mp3', '.flac', '.m4a', '.ogg']

        os.makedirs(output_folder, exist_ok=True)
        input_path = Path(input_folder)
        audio_files = []

        for ext in audio_extensions:
            audio_files.extend(input_path.rglob(f'*{ext.lower()}'))
            audio_files.extend(input_path.rglob(f'*{ext.upper()}'))

        if not audio_files:
            print(f"No audio files found in {input_folder}")
            return

        print(f"Found {len(audio_files)} audio files")

        embeddings_dict = {}
        failed_files = []
        filenames_list = []  # <<<<< This will store segment filenames in order

        for audio_file in tqdm(audio_files, desc="Extracting embeddings"):
            embedding = self.extract_embedding(str(audio_file))

            if embedding is not None:
                rel_path = str(audio_file.relative_to(input_path))
                embeddings_dict[rel_path] = embedding
                filenames_list.append(rel_path)  # Save relative path in order
            else:
                failed_files.append(str(audio_file))

        if embeddings_dict:
            combined_array = np.stack(list(embeddings_dict.values()))
            combined_file = Path(output_folder) / "combined_embeddings.npy"
            np.save(combined_file, combined_array)

            # Save the filenames list in the same output folder
            filenames_file = Path(output_folder) / "filenames.json"
            with open(filenames_file, "w") as f:
                json.dump(filenames_list, f, indent=2)

        else:
            print("⚠️ No valid embeddings to save.")

        print(f"\n✅ Extraction Summary:")
        print(f"  Successful: {len(embeddings_dict)}/{len(audio_files)}")
        print(f"  Embeddings saved to: {output_folder}")
        print(f"  Combined embeddings file: combined_embeddings.npy")
        print(f"  Filenames list saved to: filenames.json")

        if failed_files:
            print(f"\n⚠️ Failed to process {len(failed_files)} files:")
            for file in failed_files:
                print(f"  - {file}")


def run_extraction(input_folder, output_folder, model_name="speechbrain/spkrec-ecapa-voxceleb"):
    """Convenient function for running from Python (Jupyter/Colab)"""
    extractor = SpeakerEmbeddingExtractor(model_name=model_name)
    extractor.process_folder(input_folder, output_folder)

def main():
    """Command-line interface"""
    parser = argparse.ArgumentParser(description="Extract speaker embeddings using ECAPA-TDNN")
    parser.add_argument("--input_folder", type=str, required=True,
                        help="Folder containing audio files")
    parser.add_argument("--output_folder", type=str, required=True,
                        help="Folder to save embeddings")
    parser.add_argument("--model", type=str, default="speechbrain/spkrec-ecapa-voxceleb",
                        help="Model to use")

    args = parser.parse_args()
    extractor = SpeakerEmbeddingExtractor(model_name=args.model)
    extractor.process_folder(args.input_folder, args.output_folder)

# Run from script or notebook
if __name__ == "__main__":
    import sys
    if hasattr(sys, 'ps1') or 'ipykernel' in sys.modules or 'google.colab' in sys.modules:
        print("🔁 Interactive mode detected. Use run_extraction().")
        INPUT_FOLDER = "/content/segments_overlap"
        OUTPUT_FOLDER = "/content/embeddings"
        run_extraction(INPUT_FOLDER, OUTPUT_FOLDER)
    else:
        main()

In [ ]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import json  # or use plain text if needed

# Load embeddings
embeddings = np.load("/content/embeddings/combined_embeddings.npy")

# Load segment filenames in the same order as embeddings
# Example: a JSON list or plain text file, one filename per line
with open("/content/embeddings/filenames.json", "r") as f:
    segment_filenames = json.load(f)
# OR if plain text:
# with open("filenames.txt", "r") as f:
#     segment_filenames = [line.strip() for line in f.readlines()]

assert len(segment_filenames) == embeddings.shape[0], "Filenames and embeddings count mismatch"

# Tune number of speakers (clusters)
min_speakers = 2
max_speakers = 8

best_num_speakers = min_speakers
best_score = -1

for n_clusters in range(min_speakers, max_speakers + 1):
    clustering = AgglomerativeClustering(n_clusters=n_clusters, metric='cosine', linkage='average')
    labels = clustering.fit_predict(embeddings)
    if len(set(labels)) == 1:
        continue
    score = silhouette_score(embeddings, labels, metric='cosine')
    if score > best_score:
        best_score = score
        best_num_speakers = n_clusters

# Final clustering with best number of speakers
final_clustering = AgglomerativeClustering(n_clusters=best_num_speakers, metric='cosine', linkage='average')
final_labels = final_clustering.fit_predict(embeddings)

print(f"Estimated number of speakers: {best_num_speakers}")

# Map segment filenames to speaker labels
segment_to_speaker = {fname: f"Speaker_{label}" for fname, label in zip(segment_filenames, final_labels)}

# Print results
for segment, speaker in segment_to_speaker.items():
    print(f"{segment} -> {speaker}")

# Optionally save to JSON
import os
output_path = "/content/segment_speaker_labels.json"
with open(output_path, "w") as f_out:
    json.dump(segment_to_speaker, f_out, indent=2)

print(f"\nSpeaker labels saved to: {output_path}")

In [ ]:
import json
import re
from pydub import AudioSegment
import os

def parse_filename(filename):
    """Extract start, end from filename like 'segment2_1.50_3.50.wav'."""
    match = re.search(r'_(\d+\.\d+)_(\d+\.\d+)\.wav', filename)
    if match:
        return float(match.group(1)), float(match.group(2))
    else:
        raise ValueError(f"Filename format not recognized: {filename}")

def load_segments(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    segments = []
    for filename, label in data.items():
        start, end = parse_filename(filename)
        segments.append({
            'filename': filename,
            'start': start,
            'end': end,
            'label': label
        })

    # Sort by time
    return sorted(segments, key=lambda x: x['start'])

def merge_segments(segments, trim_overlap=0.5, merge_gap=0.3):
    merged = []
    if not segments:
        return merged

    current = segments[0].copy()

    for seg in segments[1:]:
        if seg['label'] == current['label']:
            # If overlapping or small gap, merge
            if seg['start'] <= current['end']:  # overlap
                current['end'] = max(current['end'], seg['end'])
            elif seg['start'] - current['end'] <= merge_gap:  # small gap
                current['end'] = seg['end']  # merge across the gap
            else:
                merged.append(current)
                current = seg.copy()
        else:
            merged.append(current)
            current = seg.copy()

    merged.append(current)

    # Optional: trim small overlaps at the end
    for i in range(1, len(merged)):
        if merged[i]['start'] < merged[i-1]['end']:
            overlap = merged[i-1]['end'] - merged[i]['start']
            trim = min(overlap / 2, trim_overlap)
            merged[i-1]['end'] -= trim
            merged[i]['start'] += trim

    return merged


def extract_audio(audio_path, merged_segments, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    audio = AudioSegment.from_file(audio_path)

    output_files = []
    for i, seg in enumerate(merged_segments):
        start_ms = int(seg['start'] * 1000)
        end_ms = int(seg['end'] * 1000)
        seg_audio = audio[start_ms:end_ms]
        out_path = os.path.join(output_dir, f"{seg['label']}_segment_{i}.wav")
        seg_audio.export(out_path, format="wav")
        output_files.append((out_path, seg['label']))
    return output_files

def save_metadata_json(merged_segments, output_dir, json_out_path):
    metadata = []
    for i, seg in enumerate(merged_segments):
        filename = os.path.join(output_dir, f"{seg['label']}_segment_{i}.wav")
        metadata.append({
            "speaker": seg["label"],
            "start": seg["start"],
            "end": seg["end"],
            "file": filename
        })

    with open(json_out_path, 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"✅ Saved metadata JSON to: {json_out_path}")

if __name__ == "__main__":
    json_path = "/content/drive/MyDrive/Colab Notebooks/New_dataset/segment_speaker_labels.json"        # your JSON file
    audio_path = "/content/drive/MyDrive/Colab Notebooks/New_dataset/preprocess.wav"          # your full audio file
    output_dir = "/content/drive/MyDrive/Colab Notebooks/Copy/merged_segments1"           # output folder for merged speaker chunks
    metadata_out = "/content/drive/MyDrive/Colab Notebooks/Copy/merged_segments1.json"

    # Step 1: Load segments
    segments = load_segments(json_path)

    # Step 2: Merge by speaker with overlap trimming
    merged_segments = merge_segments(segments, trim_overlap=0.5)

    # Step 3: Extract audio chunks and save them
    output_files = extract_audio(audio_path, merged_segments, output_dir)
    save_metadata_json(merged_segments, output_dir, metadata_out)

    print(f"✅ Done. Extracted {len(output_files)} merged chunks to: {output_dir}")
    print(f"✅ Done. Extracted {len(output_files)} merged chunks to: {metadata_out}")

In [ ]:
!pip install -U openai-whisper
!pip install torch torchvision torchaudio

In [ ]:
import whisper
import json
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")

# Load Whisper model
model = whisper.load_model("large")

# Paths
segments_folder = Path("/content/drive/MyDrive/Colab Notebooks/Copy/merged_segments")
segment_speaker_map_path = "/content/drive/MyDrive/Colab Notebooks/Copy/merged_segments.json"

# Load the mapping (list of dicts)
with open(segment_speaker_map_path, "r") as f:
    segment_speaker_map = json.load(f)

print("\n=== Transcripts with Speaker Labels and Absolute Timestamps ===\n")

# Transcribe each segment
for entry in segment_speaker_map:
    segment_filename = Path(entry["file"]).name  # get just filename
    audio_path = segments_folder / segment_filename
    speaker_label = entry["speaker"]
    segment_start_time = entry["start"]

    if not audio_path.exists():
        print(f"[WARNING] Segment not found: {audio_path}")
        continue

    result = model.transcribe(str(audio_path), task="translate")
    segments = result.get("segments", [])

    for seg in segments:
        abs_start = segment_start_time + seg['start']
        abs_end = segment_start_time + seg['end']
        text = seg['text'].strip()
        print(f"[{speaker_label}] [{abs_start:.2f}s - {abs_end:.2f}s] {text}")
    print()

In [ ]:
!pip install -q streamlit pyngrok whisper torchaudio librosa soundfile pydub speechbrain scikit-learn

In [ ]:
!pip uninstall openai-whisper
!pip install openai-whisper